### Load dataset

In [204]:
# from google.colab import drive
# drive.mount("/contetnt/drive")

In [205]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

List of activities

In [206]:
# mypath = "/content/drive/Mydrive/data"
mypath = "data"
activities = listdir(mypath)
activities

['idle', 'running', 'stairs', 'walking']

Number of activities

In [207]:
for act in activities:
    path = join(mypath, act)
    frames = listdir(path)
    print(act, ":  ", len(frames))

idle :   1039
running :   3408
stairs :   165
walking :   1850


Data visualization

In [208]:
frames = listdir(join(mypath, activities[-1]))  # "walking"
frame = pd.read_csv(join(join(mypath, activities[-1]), frames[0]))
frame

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,2.322376,3.643975,11.827356
1,1.781286,4.668694,6.148310
2,7.963115,-4.113238,1.359907
3,1.996764,-1.398214,1.163582
4,5.185841,-12.143391,4.687847
5,11.219229,-10.625467,-8.087613
6,10.888829,-13.939042,-1.273715
7,9.902418,-9.667787,-0.110133
8,8.097190,-20.934900,1.393425
9,5.655105,-11.573571,-1.106121


### Data processing

In [209]:
from scipy.fftpack import fft, rfft
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error

Features

In [210]:
features = [
    "skew_X",
    "skew_Y",
    "skew_Z",
    "kurtosis_X",
    "kurtosis_Y",
    "kurtosis_Z",
    "max_X",
    "max_Y",
    "max_Z",
    "min_X",
    "min_Y",
    "min_Z",
    "mean_X",
    "mean_Y",
    "mean_Z",
    "std_X",
    "std_Y",
    "std_Z",
    "variance_X",
    "variance_Y",
    "variance_Z",
    "median_X",
    "median_Y",
    "median_Z",
    "index_max_X",
    "index_max_Y",
    "index_max_Z",
    "index_min_X",
    "index_min_Y",
    "index_min_Z",
    "correlation_X",
    "correlation_Y",
    "correlation_Z",
    "mae_X",
    "mae_Y",
    "mae_Z",
    "rmse_X",
    "rmse_Y",
    "rmse_Z",
]

len(features)

39

Function to calculate all static features

In [211]:
def get_stat_features(frame):
    fr = frame.copy()
    features = []
    features = np.array(features)

    features = np.concatenate((features, fr.skew(axis=0).values), axis=0)
    features = np.concatenate((features, fr.kurt(axis=0).values), axis=0)
    features = np.concatenate((features, fr.max(axis=0).values), axis=0)
    features = np.concatenate((features, fr.min(axis=0).values), axis=0)
    features = np.concatenate((features, fr.mean(axis=0).values), axis=0)
    features = np.concatenate((features, fr.std(axis=0).values), axis=0)
    features = np.concatenate((features, fr.var(axis=0).values), axis=0)
    features = np.concatenate((features, fr.median(axis=0).values), axis=0)
    features = np.concatenate((features, fr.idxmax(axis=0).values), axis=0)
    features = np.concatenate((features, fr.idxmin(axis=0).values), axis=0)

    correlations = fr.corr()
    corr = np.array(
        [
            correlations["accelerometer_X"]["accelerometer_Y"],
            correlations["accelerometer_Y"]["accelerometer_Z"],
            correlations["accelerometer_Z"]["accelerometer_Z"],
        ]
    )
    features = np.concatenate((features, corr), axis=0)

    fr["mean_X"] = fr.mean(axis=0)["accelerometer_X"]
    fr["mean_Y"] = fr.mean(axis=0)["accelerometer_Y"]
    fr["mean_Z"] = fr.mean(axis=0)["accelerometer_Z"]

    mae_X = mean_absolute_error(fr["accelerometer_X"], fr["mean_X"])
    mae_Y = mean_absolute_error(fr["accelerometer_Y"], fr["mean_Y"])
    mae_Z = mean_absolute_error(fr["accelerometer_Z"], fr["mean_Z"])

    rmse_X = np.sqrt(mean_squared_error(fr["accelerometer_X"], fr["mean_X"]))
    rmse_Y = np.sqrt(mean_squared_error(fr["accelerometer_Y"], fr["mean_Y"]))
    rmse_Z = np.sqrt(mean_squared_error(fr["accelerometer_Z"], fr["mean_Z"]))

    metrics = np.array([mae_X, mae_Y, mae_Z, rmse_X, rmse_Y, rmse_Z])
    features = np.concatenate((features, metrics), axis=0)

    return features


len(get_stat_features(frame))

39

In [212]:
stat_features = pd.DataFrame(data=get_stat_features(frame), index=features)
stat_features.columns = ["values"]
stat_features

,values
skew_X,0.197811
skew_Y,0.659964
skew_Z,-1.635672
kurtosis_X,-0.543699
kurtosis_Y,1.078947
kurtosis_Z,7.158918
max_X,12.363658
max_Y,4.668694
max_Z,11.827356
min_X,-2.389413


Function to calculate all Fast Fourier Transform coefficients

In [213]:
def get_fft_features(frame):
    fft_X = np.array(preprocessing.normalize([rfft(frame["accelerometer_X"].values)]))
    fft_Y = np.array(preprocessing.normalize([rfft(frame["accelerometer_Y"].values)]))
    fft_Z = np.array(preprocessing.normalize([rfft(frame["accelerometer_Z"].values)]))

    features = np.concatenate((fft_X, fft_Y), axis=0)
    features = np.concatenate((features, fft_Z), axis=0)

    return features.T


len(get_fft_features(frame))

30

In [214]:
fft_features = pd.DataFrame(data=get_fft_features(frame), index=frame.index)
fft_features.columns = [frame.columns]
fft_features

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,0.818215,-0.922151,-0.183724
1,0.079673,0.142657,0.208943
2,-0.259434,0.026101,0.073640
3,-0.266388,0.178724,0.338499
4,-0.013780,-0.071970,-0.301204
5,-0.125891,0.140604,0.160996
6,0.083944,-0.120517,-0.028451
7,-0.016029,-0.017203,0.207865
8,-0.106537,0.033249,0.109112
9,0.003464,0.086365,0.136449


Training data preparation

In [ ]:
def class_data_prepare(class_name, class_number):
    path = join(mypath, class_name)
    X = []
    
    for item in listdir(path):
        frame = pd.read_csv(join(path, item))
        frame = frame.values
        frame = frame.flatten()
        X.append(frame)
        
    y = [class_number]*len(X)
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y

### SVM Classifier

In [215]:
from sklearn import svm

classifier = svm.SVC(decision_function_shape="ovo", kernel="rbf", gamma=0.005, probability=True)

In [ ]:
classifier.fit(X_train, y_train)